### Imports

In [33]:
import numpy as np
import random 
import math
import time

### Functions

- **Distance Function** : Returns the distance between two points if its co-ordinates are given

In [34]:
def find_distance(x1,x2,y1,y2):
    return math.sqrt((x1-x2)**2+(y1-y2)**2)

- **Cost Function** : Returns the cost matrix where Cij denotes the amount of water transported from Si to Dj multiplied with the distance between them

In [35]:
def find_cost(supply_coords,demand_coords):
    cost = np.zeros((len(supply_coords),len(demand_coords)))
    for i in range(len(supply_coords)):
        for j in range(len(demand_coords)):
            cost[i,j] = find_distance(supply_coords[i][0],demand_coords[j][0],supply_coords[i][1],demand_coords[j][1])
    return cost

- **Bubble Sort** : Returns the sorted list in ascending order based on their cost

In [36]:
def bubble_sort(arrx,cost):
    parents = []
    for i in range(len(arrx)):
        parents.append(arrx[i][1])
    swapped = True
    while swapped:
        swapped = False
        for i in range(len(arrx) - 1):
            if arrx[i][0] > arrx[i + 1][0]:
                arrx[i], arrx[i + 1] = arrx[i + 1], arrx[i]
                parents[i],parents[i+1] = parents[i+1],parents[i]
                swapped = True
    final = []
    for i in range(len(parents)):
        final.append([np.sum(cost*parents[i]),parents[i]])
    return final

- **Initialize Parent Generation** : Initiates 'population' number of members in the parent generation randomly 

In [37]:
def init_Parent_Gen(population,num_supply,num_demand,source,dest,cost):
    # We have used the method given in Page 281 of Genetic Algorithms and Engineering Design Book.
    pi = []
    for i in range(1,num_supply*num_demand+1):
        pi.append(i)  
    parents= []
    
    # This function generates population number of species in the initial generation
    for p in range(population):
        X = np.zeros((num_supply,num_demand))
        s = source.copy()
        d = dest.copy()
        test = pi.copy()              # A list containing values from 1 to number of grids in the transportation matrix is made.
        while(len(test)!=0):
            k = random.choice(test)   # A random number is selected from this list
            i = int(((k-1)/len(d)))   # Using the following formula and the random number, we find row and column index
            j = ((k-1)%len(d)) 
            X[i,j] = min(s[i],d[j])   # We assign the minimum value of the corresponding source value or demand value to that box
            s[i] = s[i] - X[i,j]      # The source and demand are updated so that constraints are satisfied
            d[j] = d[j] - X[i,j]
            test.remove(k)            # The random number is removed from the list
        parents.append([np.sum(cost*X),X])   
    return parents

- **Remove copies in the generation**

In [38]:
def remove_copies(parents):
    
    # This function assumes that it gets a sorted list in the format[[cost,parents],...]
    x = []
    final = []
    
    # If the previous cost and current cost is same, that parent is not selected to the list returned.
    for i in range(len(parents)):
        x.append(parents[i][0])
        if i==0:
            final.append(parents[i])
        if i!=0 and x[i]!=x[i-1]:
            final.append(parents[i])
    return final

- **Roulette's selection** : Probabilistic approach to select parents

In [39]:
def selection(parents):
    # F is the total fitness
    F = 0
    for i in range(len(parents)):
        F = F+ parents[i][0]
        
    # prb contains the probability of selecting the points
    prb = []
    for i in range(len(parents)):
        prb.append((F- parents[i][0])/(F*(len(parents)-1)))
        
    # cum_prb is the cumulative probability
    cum_prb = []
    for i in range(len(prb)):
        if i==0:
            cum_prb.append(prb[i])
        else:
            cum_prb.append(cum_prb[-1]+prb[i])
            
    # we generate a random number r and find the interval of cumulative probability 
    # in which the r lies
    r = random.random()
    error = 1
    e_ind = 1
    for i in range(len(cum_prb)):
        if error>abs(cum_prb[i]-r):
            error=abs(cum_prb[i]-r)
            e_ind = i
    return parents[e_ind][1]

- **Crossover**

In [40]:
def crossover(p,q,num_supply,num_demand,cost):
    
    # Refer paper of the book mentioned earlier to see mathematical method used here.
    
    # We define and make D and R matrices according to their formulas 
    D = np.zeros((num_supply,num_demand))
    R = np.zeros((num_supply,num_demand))
    for i in range(num_supply):
        for j in range(num_demand):
            D[i,j] = int((p[i,j]+q[i,j])/2)
            R[i,j] = (p[i,j]+q[i,j])%2
    
    # now we have to split R into R1 and R2. The following variables helps to keep a 
    # check on the horizontal and vertical sum constraints of R1 and R2
    horP_sum = np.sum(p,axis=1)
    verP_sum = np.sum(q,axis=0)
    R_dash = []
    R_costs = []
    
    # A modified method used for initializing parent generation is used to find R1 and R2 satisfying the constraints
    # This loop breaks when R1 and R2 are found. In case we are not able to split R into two, the loop runs for 100 times.
    for i in range(100):
        A = np.zeros((num_supply,num_demand))
        test = []
        s = np.sum(R/2,axis=1)
        d = np.sum(R/2,axis=0)
        for i in range(1,len(s)*len(d)+1):
            test.append(i)
        while(len(test)!=0):
            k = random.choice(test)
            i = int(((k-1)/len(d)))
            j = ((k-1)%len(d))
            A[i,j]=0
            if s[i]!=0 and d[j]!=0:
                    A[i,j] = 1
                    s[i] = s[i]-1
                    d[j]= d[j]-1
            if np.sum(D+A,axis=1)[i]>horP_sum[i]: #Additional checks so that R1 and R2's constraints are satisfied
                if np.sum(D+A,axis=0)[j]>verP_sum[j]:
                    A[i,j] = 0
            test.remove(k)
        if np.sum(cost*A) not in R_costs:
            R_costs.append(np.sum(cost*A))
            R_dash.append(A)
        if len(R_dash)==2:
            break
            
    # If flag is 0, that means we were not able to split R into 2, and hence only one offspring is formed.
    # If flag is 1, two offsprings are generated
    flag = 0
    X1 = D+R_dash[0]
    if len(R_dash)>1:
        X2 = D+R_dash[1]
        flag = 1
        return X1,X2,flag
    else:
        return X1,X1,flag

- **Mutation** 

In [41]:
def mutate(a):
    # Refer paper of the book mentioned earlier to see mathematical method used here.
    
    # we select random number of rows and columns from the parent
    # For this the number of rows and columns are first figured out.
    n_rows, n_cols = random.randint(2,a.shape[0]),random.randint(2,a.shape[1])
    
    # Now the row and column indexes are figured out
    row = []
    col = []
    while len(row)<n_rows:
        x = random.randint(0,a.shape[0]-1)
        if x not in row:
            row.append(x)
    while len(col)<n_cols:
        x = random.randint(0,a.shape[1]-1)
        if x not in col:
            col.append(x)
    row.sort()
    col.sort()
    
    # The values corresponding to the row and column indexes is made into A
    A = np.zeros((n_rows,n_cols))
    
    # Here we again employ the method used in init_parents to find different combinations of the A matrix such that the
    # row sum and column sum remain the same.
    
    s = np.sum(a[np.ix_(row,col)],axis=1)
    d = np.sum(a[np.ix_(row,col)],axis=0)   
    test = []
    for i in range(1,len(s)*len(d)+1):
        test.append(i)    
    while(len(test)!=0):
        k = random.choice(test)
        i = int(((k-1)/len(d)))
        j = ((k-1)%len(d)) 
        A[i,j] = min(s[i],d[j])
        s[i] = s[i] - A[i,j]
        d[j] = d[j] - A[i,j]
        test.remove(k) 
    
    # Finally the Mutated matrix A is combined with the original matrix and returned
    row_itr = 0
    col_itr = 0
    for i in row:
        for j in col:
            a[i,j] = A[row_itr,col_itr]            
            col_itr = col_itr+1
        row_itr = row_itr+1
        col_itr=0
    return a

- **CROSSOVER MAIN LOOP** : Give offsprings from nth generation using crossover function

In [42]:
def do_cross(copy_parents,cross_num,num_supply,num_demand,cost,source,dest):
    
    # This function controls the crossover loop
    offsprings = []
    parents = add_or_remove(copy_parents,1,cost)
    
    # n is the number of parents to be selected for crossover calculated using cross_num*population
    n = int(cross_num*len(copy_parents))
    while n!=1 and n!=0:
        a = selection(parents)    # selection done using Roulette's wheel
        b = selection(parents)
        if np.sum(cost*a)!=np.sum(cost*b):
            off1 = np.zeros((num_supply,num_demand))
            off2 = np.zeros((num_supply,num_demand))
            off1,off2,flag = crossover(a,b,num_supply,num_demand,cost)
            temp_hor = 0
            temp_vert = 0
            
            # flag==1 shows that two offsprings are coming
            if flag==1:
                horOff1 = np.sum(off1,axis=1)
                vertOff1 = np.sum(off1,axis=0)
                horOff2 = np.sum(off2,axis=1)
                vertOff2 = np.sum(off2,axis=0)
                
                # extra checks to see if the offsprings satisfy the constraints
                for i in range(num_supply):
                    if source[i]==horOff1[i] and source[i]==horOff2[i]:
                        temp_hor = temp_hor+1
                for i in range(num_demand):
                    if dest[i]==vertOff1[i] and dest[i]==vertOff2[i]:
                        temp_vert = temp_vert+1
                if temp_hor==num_supply and temp_vert==num_demand:
                    offsprings.append(off1)
                    offsprings.append(off2)
                n=n-2
            else:
                horOff1 = np.sum(off1,axis=1)
                vertOff1 = np.sum(off1,axis=0)
                for i in range(num_supply):
                    if source[i]==horOff1[i]:
                        temp_hor = temp_hor+1
                for i in range(num_demand):
                    if dest[i]==vertOff1[i]:
                        temp_vert = temp_vert+1
                if temp_hor==num_supply and temp_vert==num_demand:
                    offsprings.append(off1)
                n=n-1
    return offsprings

- **Add/Remove cost** : Converts list of parents from [[cost,parent]..] to [[parent]..] and vice-versa

In [43]:
def add_or_remove(parents,flag,cost):
    p = []
    if flag==1:
        for i in range(len(parents)):
            p.append([np.sum(cost*parents[i]),parents[i]])
    if flag==0:
        for i in range(len(parents)):
            p.append(parents[i][1])
    return p

- **MUTATION MAIN LOOP** : Give offsprings from nth generation using mutation function

In [44]:
def do_mutate(copy_parents,mut_num,cost,num_supply,num_demand,source,dest):
    
    # n is the number of parents to be selected for mutation calculated using mut_num*population
    n = int(mut_num*len(copy_parents))
    parents = add_or_remove(copy_parents,1,cost)
    offsprings = []
    temp_hor = 0
    temp_vert = 0
    
    # This loop adds the offsprings to a list if it satisfies constraints
    while n!=0:
        a = selection(parents)
        off1 = mutate(a)
        horOff1 = np.sum(off1,axis=1)
        vertOff1 = np.sum(off1,axis=0)
        for i in range(num_supply):
            if source[i]==horOff1[i]:
                temp_hor = temp_hor+1
        for i in range(num_demand):
            if dest[i]==vertOff1[i]:
                temp_vert = temp_vert+1
        if temp_hor==num_supply and temp_vert==num_demand:
            offsprings.append(off1)
        n = n-1
    return offsprings

#### INPUT 

In [45]:
source = [8,4,12,6]
dest = [3,5,10,7,5]
supply_coords = [[7., 6.],[2., 7.],[1., 6.],[6., 8.]]
demand_coords = [[3., 9.],[4., 6.],[5., 7.],[5., 3.],[2., 9.]]
# source = [13,12]
# dest = [9, 16]
# supply_coords = [[7., 6.],[2., 5.]]
# demand_coords = [[3., 9.],[4., 6.]]
##############
# source = [3,3]
# dest = [2, 4]
# supply_coords = [[7., 6.],[8.338, 3.513]]
# demand_coords = [[7., 5.],[4., 6.]]
population = 100
num_supply = len(source)
num_demand = len(dest)
cross_num = 0.4
mutate_num = 0.2
num_gen = 1000
convergence_no = 25

#### Main loop

In [46]:
start = time.time()
cost = find_cost(supply_coords,demand_coords)
parents = init_Parent_Gen(population,num_supply,num_demand,source,dest,cost)
parents = bubble_sort(parents,cost)
parents= remove_copies(parents)
no = len(parents)
p = parents.copy()
itr = 0
temp = []
endFinder = 10000  # This value keeps updating itself with the most minimum cost found till now. 
endNum =0          # endNum gives the number of occurances of endFinder
c=[]
m=[]


for i in range(num_gen):

    temp = []
    # crossover and mutation
    c = add_or_remove(do_cross(add_or_remove(p,0,cost),cross_num,num_supply,num_demand,cost,source,dest),1,cost)
    m = add_or_remove(do_mutate(add_or_remove(p,0,cost),mutate_num,cost,num_supply,num_demand,source,dest),1,cost)
    
    # perform elitist selection
    temp = c+m+p

    temp = bubble_sort(temp,cost)
    temp = remove_copies(temp)
    p = []
    
    
    p = temp[:population]

    itr = itr+1
    if endFinder>p[0][0]:
        endFinder=p[0][0]
        if endNum==0:
            endNum= endNum+1
        else:
            endNum=0
    elif endFinder==p[0][0]:
        endNum= endNum+1
    
    # if endNum occurs convergence number times, then we can say that it has converged.
        if (endNum-1)==convergence_no:
            print("FINAL ANSWER" ,p[0][0],p[0][1],np.sum(p[0][1]*cost))
            break

    print("Iteration",i,":",p[0][0],"Minimum cost: " ,endFinder)

end = time.time()

Iteration 0 : 83.8306898727 Minimum cost:  83.8306898727
Iteration 1 : 83.8306898727 Minimum cost:  83.8306898727
Iteration 2 : 87.5004099013 Minimum cost:  83.8306898727
Iteration 3 : 87.5004099013 Minimum cost:  83.8306898727
Iteration 4 : 134.626970868 Minimum cost:  83.8306898727
Iteration 5 : 88.7466311543 Minimum cost:  83.8306898727
Iteration 6 : 88.7466311543 Minimum cost:  83.8306898727
Iteration 7 : 88.5591490605 Minimum cost:  83.8306898727
Iteration 8 : 85.4162785971 Minimum cost:  83.8306898727
Iteration 9 : 84.9723894369 Minimum cost:  83.8306898727
Iteration 10 : 84.9723894369 Minimum cost:  83.8306898727
Iteration 11 : 84.9723894369 Minimum cost:  83.8306898727
Iteration 12 : 84.9723894369 Minimum cost:  83.8306898727
Iteration 13 : 84.9723894369 Minimum cost:  83.8306898727
Iteration 14 : 84.0218298725 Minimum cost:  83.8306898727
Iteration 15 : 84.0218298725 Minimum cost:  83.8306898727
Iteration 16 : 84.0218298725 Minimum cost:  83.8306898727
Iteration 17 : 83.814624

### Final Output

Minimized cost

In [47]:
print(p[0][0])

83.2090729593


Solution matrix

In [48]:
print(p[0][1])

[[ 0.  0.  4.  4.  0.]
 [ 3.  0.  0.  0.  1.]
 [ 0.  5.  0.  3.  4.]
 [ 0.  0.  6.  0.  0.]]


Time Taken

In [49]:
print(end-start,"sec")

6.689572334289551 sec
